# Predicting flight arrivals

## A simple binary classification example

In this notebook, we'll explore *binary classification* - a form of machine learning in which we train a model to predict which to two categories, or *classes* something belongs. Specifically, we'll use details of flight records from the US department of transportation to train a model that predicts whether a flight will arrive late or on-time.

Let's start by loading and viewing the data.

In [ ]:
import pandas as pd

df_flights = pd.read_csv('data/flights.csv')
df_flights.head()

The dataset contains observations of US domestic flights in 2013, and consists of the following fields:

- **Year**: The year of the flight (all records are from 2013)
- **Month**: The month of the flight
- **DayofMonth**: The day of the month on which the flight departed
- **DayOfWeek**: The day of the week on which the flight departed - from 1 (Monday) to 7 (Sunday)
- **Carrier**: The two-letter abbreviation for the airline.
- **OriginAirportID**: A unique numeric identifier for the departure aiport
- **OriginAirportName**: The full name of the departure airport
- **OriginCity**: The departure airport city
- **OriginState**: The departure airport state
- **DestAirportID**: A unique numeric identifier for the destination aiport
- **DestAirportName**: The full name of the destination airport
- **DestCity**: The destination airport city
- **DestState**: The destination airport state
- **CRSDepTime**: The scheduled departure time
- **DepDelay**: The number of minutes departure was delayed (flight that left ahead of schedule have a negative value)
- **ArrDelay15**: A binary indicator that departure was delayed by more than 15 minutes (and therefore considered "late")
- **CRSArrTime**: The scheduled arrival time
- **ArrDelay15**: A binary indicator that arrival was delayed by more than 15 minutes (and therefore considered "late")
- **Cancelled**: A binary indicator that the flight was cancelled

### Clean the data

Before we can train a model, we need to clean and explore the data. First. let's see if there are any missing values.

In [ ]:
df_flights.isnull().sum()

It looks like there are some null "late departure" indicators. Departures are considered late if the delay is 15 minutes or more, so let's see the delays for the ones with a null late indicator:

In [ ]:
df_flights[df_flights.isnull().any(axis=1)][['DepDelay','DepDel15']]

We can't see them all in this display, but it looks like they may all have delay of 0. Let's check by looking at the summary statistics for these records:

In [ ]:
df_flights[df_flights.isnull().any(axis=1)].DepDelay.describe()

The min, max, and mean are all 0; so it seems that none of these were actually *late* departures. Let's replace the missing **DepDel15** indicator with a 0 and confirm there are no more missing values.

In [ ]:
df_flights.DepDel15 = df_flights.DepDel15.fillna(0)
df_flights.isnull().sum()

While the dataset contains many numeric values, most of these represent *categorical* variables (for example, months, days, airports, ...). The only actual numeric value that represents a quantity we can measure is the departure delay.

Let's take a look at the distribution and summary statistics for the *DepDelay* column to get a sense of typical departure delays.

> **Note**: We'll want to do this again, so we'll define a function that we can use with any column.

In [ ]:
# Function to show summary stats and distribution for a column
def show_distribution(var_data):
    from matplotlib import pyplot as plt
    %matplotlib inline

    # Get statistics
    min_val = var_data.min()
    max_val = var_data.max()
    mean_val = var_data.mean()
    med_val = var_data.median()
    mod_val = var_data.mode()[0]

    print(var_data.name,'\nMinimum:{:.2f}\nMean:{:.2f}\nMedian:{:.2f}\nMode:{:.2f}\nMaximum:{:.2f}\n'.format(min_val,
                                                                                            mean_val,
                                                                                            med_val,
                                                                                            mod_val,
                                                                                            max_val))

    # Create a figure for 2 subplots (2 rows, 1 column)
    fig, ax = plt.subplots(2, 1, figsize = (10,4))

    # Plot the histogram   
    ax[0].hist(var_data)
    ax[0].set_ylabel('Frequency')

    # Add lines for the mean, median, and mode
    ax[0].axvline(x=min_val, color = 'gray', linestyle='dashed', linewidth = 2)
    ax[0].axvline(x=mean_val, color = 'cyan', linestyle='dashed', linewidth = 2)
    ax[0].axvline(x=med_val, color = 'red', linestyle='dashed', linewidth = 2)
    ax[0].axvline(x=mod_val, color = 'yellow', linestyle='dashed', linewidth = 2)
    ax[0].axvline(x=max_val, color = 'gray', linestyle='dashed', linewidth = 2)

    # Plot the boxplot   
    ax[1].boxplot(var_data, vert=False)
    ax[1].set_xlabel('Value')

    # Add a title to the Figure
    fig.suptitle(var_data.name)

    # Show the figure
    fig.show()

# Call the function for the DepDelay field
show_distribution(df_flights['DepDelay'])

The departure delays are measured in minutes, with negative values indicating early departures. The distribution of the data is extremely *left-skewed*; in other words, the bulk of the data is at the lower end. The *mean* value is quite low, which is what you might expect (most flights depart on time, with some slight delays and some slightly early departures). However, there are a few extremely high values that have the effect of increasing the *mean* delay. On the boxplot, you can clearly see a lot of **o** markers that indicate statistical *outliers* - values that are abnormally outside of the distribution of most of the data - in this case, values that are unusally high.

Often, outliers represent atypical cases that might bias a model. Let's trim the data so that we include only rows where the departure delay is within the 1st and 90th percentile.

In [ ]:
# Trim outliers for DepDelay based on 1% and 90% percentiles
DepDelay_01pcntile = df_flights.DepDelay.quantile(0.01)
DepDelay_90pcntile = df_flights.DepDelay.quantile(0.90)
df_flights = df_flights[df_flights.DepDelay < DepDelay_90pcntile]
df_flights = df_flights[df_flights.DepDelay > DepDelay_01pcntile]

# View the revised distributions
show_distribution(df_flights['DepDelay'])

OK, so now we have a representative set of flights based on departure delays, let's compare how many arrived late and how many arrived on time.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
    
label_counts = df_flights['ArrDel15'].value_counts()
fig = plt.figure(figsize=(6,6)) 
ax = fig.gca() 
label_counts.plot.bar(ax=ax) 
ax.set_title('Late Arrival Counts') 
ax.set_xlabel('Late?') 
ax.set_ylabel('Flights')
plt.show()

The dataset contains many more examples of flights that were on time than flights that were late. In other words, the dataset is *imbalanced*. While for travellers, this is good news (many more flight arrive on time than are late), from the perspective of training a machine learning model it may bias the model. It's generally better to train a classification model with a more or less even number of cases for each class.

There are two main approaches we could use to balance the data:

- *Undersample* the majority case - in other words, eliminate some of the "on-time" cases to match the number of "late" cases. This is a legitimate strategy if it will leave enough cases with which to train a model effectively.
- *Oversample* the minority case - in other words create more "late" cases to match the number of "on-time" cases. We could do this simply by duplicating "late" cases; essentially double (or triple) counting them, or by generating new cases that are statistically similar to existing "late" cases (commonly referred to as Synthetic Minority Oversampling Technique, or SMOTE).

In this case, the are over 25,000 late cases; so if we undersample on-time cases to randomly select the same number of late cases,that gives us a balanced dataset with over 50,000 cases; which should be enough to train a model with. 

In [ ]:
# Get the late cases
df_late = df_flights[df_flights['ArrDel15'] != 0]

# get a sample of the same number of on-time cases
df_ontime = df_flights[df_flights['ArrDel15'] == 0].sample(len(df_late))

# Combine them to create a balanced sample
df_balanced = df_late.append(df_ontime)

# Plot the counts of each class
label_counts = df_balanced['ArrDel15'].value_counts()
fig = plt.figure(figsize=(6,6)) 
ax = fig.gca() 
label_counts.plot.bar(ax=ax) 
ax.set_title('Late Arrival Counts') 
ax.set_xlabel('Late?') 
ax.set_ylabel('Flights')
plt.show()

OK, so now we have a balanced dataset, let's explore the data and see if we can determine some relationships.

We know that the dataset contains one numeric feature (*DepDelay*), so a good starting point might be to compare the distribution of departure delays for flights that arrived on time against flights that arrived late.

In [ ]:
fig = plt.figure(figsize=(6,6)) 
ax = fig.gca()
df_balanced.boxplot(column='DepDelay', by='ArrDel15', ax=ax)
ax.set_title('Departure Delay Distributions')
ax.set_ylabel("Flights")
ax.set_xlabel("Late?")
plt.show()

Unsurprisingly, the median departure delay for flights that arrived late is higher than that of on-time arrivals - indicating that there may be a relationship between late departures and late arrivals. In other words, the length of departure delay may be a good indicator of whether or not the flight will arrive on time.

Now let's look at some of the *categorical* features in our dataset, and see if we can determine any relationships between them and whether or not flights arrive late or on-time.

In [ ]:
features = ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'Carrier', 'OriginAirportID', 'DestAirportID', 'OriginCity', 'DestCity', 'OriginState', 'DestState']

for col in features:

# Group by feature
    group = df_balanced.groupby(df_balanced[col])
    df_grouped = pd.DataFrame(group['ArrDel15'].sum()).sort_values(col)

    # Plot the counts of each class
    fig = plt.figure(figsize=(16,6)) 
    ax = fig.gca() 
    df_grouped.plot.bar(ax=ax) 
    ax.set_title('Late Arrivals by {}'.format(col)) 
    ax.set_xlabel(col) 
    ax.set_ylabel('Late Arrivals')

plt.show()

The dataset only includes one year, so that's not particularly useful. Some of the other columns however do show some variability in the number of late arrivals, so they may help predict whether or not a flight will be late or on-time.

There are too many airport codes to make much sense of the bar charts for those, and thinking about it logically, a more realistic way to determine the relationship between airports and late arrivals might be to consider each combination of otigin and destination airports - in other words *routes*. Let's explore that intuition that by grouping the origin and destination route combinations and counting the number of late arrivals for each. There's an enormous number of combinations, so we'll just look at a sample of 100 of them.

In [ ]:
# Create a routes table
routes  = pd.Series(df_balanced['OriginAirportName'] + ' > ' + df_balanced['DestAirportName'])
df_routes = pd.concat([df_balanced, routes.rename("Route")], axis=1)

# Group by routes
route_group = df_routes.groupby(df_routes['Route'])
df_grouped =  pd.DataFrame(route_group['ArrDel15'].sum()).sample(100).sort_values('Route')

# Plot the counts of each class
fig = plt.figure(figsize=(16,6)) 
ax = fig.gca() 
df_grouped.plot.bar(ax=ax) 
ax.set_title('Late Arrivals by Route') 
ax.set_xlabel('Route') 
ax.set_ylabel('Late Arrivals')
plt.show()

The results show that some routes have a high number of late arrivals, while others have none - so  the route might help predict whether a flight will be late or on-time. 

So far, we haven't considered the scheduled departure and arrival times (*CRSDepTime* and *CRSArrTime*). These values are integer numbers that represent the time in 24-hour clock notation (so for example, 710 represents 7:10am, and 2108 represents 9:08pm). Comparing times for every minute may result in two many categories, but what if we compare delays for each hour of the day (0 to 23)?

In [ ]:
time_cols = ['CRSDepTime', 'CRSArrTime']
for col in time_cols:

# Group by feature
    group = df_balanced.groupby(round(df_balanced[col]/100, 0))
    df_grouped = pd.DataFrame(group['ArrDel15'].sum()).sort_values(col)

    # Plot the counts of each class
    fig = plt.figure(figsize=(16,6)) 
    ax = fig.gca() 
    df_grouped.plot.bar(ax=ax) 
    ax.set_title('Late Arrivals by {}'.format(col)) 
    ax.set_xlabel(col) 
    ax.set_ylabel('Late Arrivals')

plt.show()

It looks like there's a pattern for both scheduled departure and arrival times. In which flights scheduled to depart or arrive in the late afternoon or evening (between 2:00pm and 8:00pm) tend to have the more late arrivals than flights scheduled in the early hours of the morning.

All of this exploratory data analysis provides us with some ideas of which features are likely to help predict whether a flight will be late or on-time. Let's add columns for the derived features we've identified as potentially useful (route, scheduled departure hour, and scheduled arrival hour) to the dataset (this kind of creation of new features is often referred to as *feature engineering*), and then select the subset of features we want to use to train a classification model.

In [ ]:
# Create a copy of the balanced dataframe
df_flight_data = df_balanced.copy()

# Add the scheduled hour columns
for col in time_cols:
    new_col = pd.Series(round(df_flight_data[col]/100, 0))
    df_flight_data = pd.concat([df_flight_data, new_col.rename(col + '_Hour')], axis=1)

# Add the route column
df_flight_data = pd.concat([df_flight_data, routes.rename("Route")], axis=1)

# specify the features we want to use, and the label we want to predict
features_and_label = ['DepDelay', 'Month', 'DayofMonth', 'DayOfWeek', 'Carrier', 'Route', 'CRSDepTime_Hour', 'CRSArrTime_Hour', 'ArrDel15']

df_flight_data = df_flight_data[features_and_label]
df_flight_data


In [ ]:
# Train the model
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression

# Separate features and labels
features = ['DepDelay', 'Month', 'DayofMonth', 'DayOfWeek', 'Carrier', 'Route', 'CRSDepTime_Hour', 'CRSArrTime_Hour']
label = 'ArrDel15'
X, y = df_flight_data[features].values, df_flight_data[label].values

# Split data 70%-30% into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Normalize the numeric columns
numeric_features = [0]
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

# Encode the categorical features
categorical_features = [1,2,3,4,5,6,7]
categorical_transformer = Pipeline(steps=[
    ('ordinal_encode', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)]
    )

# Create preprocessing and training pipeline
reg = 0.01
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', LogisticRegression(C=1/reg, solver="liblinear"))])


# fit the pipeline to train a model on the training set, including the pre-processing steps
model = pipeline.fit(X_train, y_train)
print (model)

In [ ]:
predictions = model.predict(X_test)
print('Predicted labels: ', predictions)
print('Actual labels:    ' ,y_test)

In [ ]:
from sklearn.metrics import accuracy_score

print('Accuracy: ', accuracy_score(y_test, predictions))

In [ ]:
from sklearn. metrics import classification_report

print(classification_report(y_test, predictions))

In [ ]:
from sklearn.metrics import precision_score, recall_score

print("Overall Precision:",precision_score(y_test, predictions))
print("Overall Recall:",recall_score(y_test, predictions))

In [ ]:
from sklearn.metrics import confusion_matrix

# Print the confusion matrix
cm = confusion_matrix(y_test, predictions)
print (cm)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

classes = ['on-time', 'late']
plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)
plt.xlabel("Predicted Class")
plt.ylabel("Actual Class")
plt.show()

In [ ]:
y_scores = model.predict_proba(X_test)
print(y_scores)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline

# calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])

# plot ROC curve
fig = plt.figure(figsize=(6, 6))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Create preprocessing and training pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', GradientBoostingClassifier())])

# fit the pipeline to train a model on the training set
model = pipeline.fit(X_train, (y_train))

# Evaluate model
predictions = model.predict(X_test)
print('Accuracy: ', accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))

# Plot confusion matrix
cm = confusion_matrix(y_test, predictions)
plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)
plt.xlabel("Predicted Class")
plt.ylabel("Actual Class")
plt.show()

# plot ROC curve
y_scores = model.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 6))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Create preprocessing and training pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', GradientBoostingClassifier(learning_rate=0.5, n_estimators=100))])

# fit the pipeline to train a model on the training set
model = pipeline.fit(X_train, (y_train))

# Evaluate model
predictions = model.predict(X_test)
print('Accuracy: ', accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))

# Plot confusion matrix
cm = confusion_matrix(y_test, predictions)
plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
plt.colorbar()
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)
plt.xlabel("Predicted Class")
plt.ylabel("Actual Class")
plt.show()

# plot ROC curve
y_scores = model.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 6))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))